In [1]:
!pip install imutils
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np


In [2]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import matplotlib as mt
train_datagen=ImageDataGenerator()

test_datagen= ImageDataGenerator(rescale=1./255)


In [3]:
batch_size=10
train_data=train_datagen.flow_from_directory(
'data/train/',
target_size=(150,150),
batch_size=batch_size,
class_mode='binary')


Found 1315 images belonging to 2 classes.


In [4]:
test_data=test_datagen.flow_from_directory(
'data/test/',
target_size=(150,150),
batch_size=batch_size,
class_mode='binary')

Found 194 images belonging to 2 classes.


In [5]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',
                             verbose=0,save_best_only=True,mode='auto')

In [8]:
history=model.fit_generator(
                            train_data,steps_per_epoch=50//batch_size,epochs=10,
                             validation_data=test_data,validation_steps=40//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
5/5 [==============================] - 3s 643ms/step - loss: 7.4463 - acc: 0.6000 - val_loss: 7.6620 - val_acc: 0.5750
Epoch 2/10
5/5 [==============================] - 3s 627ms/step - loss: 7.6703 - acc: 0.4800 - val_loss: 7.6707 - val_acc: 0.4750
Epoch 3/10
5/5 [==============================] - 3s 661ms/step - loss: 7.6685 - acc: 0.5000 - val_loss: 7.6642 - val_acc: 0.5500
Epoch 4/10
5/5 [==============================] - 3s 626ms/step - loss: 7.6685 - acc: 0.5000 - val_loss: 7.6685 - val_acc: 0.5000
Epoch 5/10
5/5 [==============================] - 3s 636ms/step - loss: 7.6685 - acc: 0.5000 - val_loss: 7.6554 - val_acc: 0.6500
Epoch 6/10
5/5 [==============================] - 3s 675ms/step - loss: 7.6756 - acc: 0.4200 - val_loss: 7.6795 - val_acc: 0.3750
Epoch 7/10
5/5 [==============================] - 3s 656ms/step - loss: 7.6580 - acc: 0.6200 - val_loss: 7.6817 - val_acc: 0.3500
Epoch 8/10
5/5

In [9]:
model.save('FaceMaskDetection.h5')

In [10]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model
model = tf.keras.models.load_model('FaceMaskDetection.h5')

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

KeyboardInterrupt: 